# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet') # download for lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, fbeta_score, recall_score, classification_report, accuracy_score, precision_score, make_scorer, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from workspace_utils import active_session
import pickle
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dagus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dagus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql("SELECT * FROM disaster_data",engine)
# df.head()
X = df.message.values
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

In [6]:
# Take only 10% of data for fitting using GridSearchCV so that it's faster
# train = np.concatenate((X_train, y_train), axis=1)
# X_train_d = X_train[:,np.newaxis]
# train = np.hstack((X_train, y_train))

# train_sample = train.sample(frac=0.1, replace=False, random_state=None, axis=0)

number_of_rows = X_train.shape[0]

# np.random.seed(123) # uncomment if we want repeatable results
random_rows = np.random.choice(number_of_rows, size=int(0.10*number_of_rows), replace=False)

X_train_sample = X_train[random_rows] # this is numpy ndarray
y_train_sample = y_train.iloc[random_rows,:] # this is pandas df

In [7]:
# pipeline.fit(X_train_sample,y_train_sample) # for now, do only with a subset, to compare to tuned model, which needs subset to run in reasonable time

pipeline.fit(X_train,y_train)

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dagus\Anaconda3

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
predicted = pipeline.predict(X_test)

In [9]:
# predicted.shape

In [10]:
# y_test.iloc[:,0].value_counts()

In [11]:
# f1_score = fbeta_score(y_test,predicted,beta=1)
# precision = fbeta_score(y_test,predicted,beta=0)
# recall =  recall_score(y_test,predicted)

# for i in range(y_test.shape[1]):
#     print(classification_report(y_test.iloc[:,i],predicted[:,i]))

# print(classification_report(y_test.values,predicted,target_names=Y.columns.values))

category_names = Y.columns.values
for i, c in enumerate(category_names): 
    print(c)
#     if i==1:
#         test=classification_report(y_test.iloc[:,i], predicted[:,i],output_dict=True)
    print(classification_report(y_test.iloc[:,i], predicted[:,i])) # the averages given in Udacity workspace are weighted; on my computer, I get macro avg and accuracy in addition
#     print('Accuracy: ', accuracy_score(y_test.iloc[:,i],predicted[:,i]), '\n')


related
              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1508
           1       0.86      0.92      0.88      4999

    accuracy                           0.82      6507
   macro avg       0.74      0.70      0.72      6507
weighted avg       0.80      0.82      0.81      6507

Accuracy:  0.8158905793760566 

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5402
           1       0.81      0.47      0.59      1105

    accuracy                           0.89      6507
   macro avg       0.85      0.72      0.76      6507
weighted avg       0.88      0.89      0.88      6507

Accuracy:  0.8905793760565545 

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6470
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.5

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy:  0.9943138158905793 

aid_related
              precision    recall  f1-score   support

           0       0.75      0.86      0.80      3823
           1       0.75      0.60      0.66      2684

    accuracy                           0.75      6507
   macro avg       0.75      0.73      0.73      6507
weighted avg       0.75      0.75      0.75      6507

Accuracy:  0.7508836637467343 

medical_help
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5987
           1       0.52      0.09      0.16       520

    accuracy                           0.92      6507
   macro avg       0.72      0.54      0.56      6507
weighted avg       0.89      0.92      0.89      6507

Accuracy:  0.9207007837713231 

medical_products
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6177
           1       0.71      0.13      0.22       330

    accuracy                           0.95 

Accuracy:  0.9814046411556785 

other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6167
           1       0.48      0.05      0.09       340

    accuracy                           0.95      6507
   macro avg       0.72      0.52      0.53      6507
weighted avg       0.93      0.95      0.93      6507

Accuracy:  0.9475948978023667 

direct_report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      5261
           1       0.73      0.32      0.45      1246

    accuracy                           0.85      6507
   macro avg       0.80      0.65      0.68      6507
weighted avg       0.83      0.85      0.82      6507

Accuracy:  0.8477024742584909 



In [12]:
metrics = dict()
for i, c in enumerate(category_names): 
    print(c)
    metrics[c] = precision_recall_fscore_support(y_test.iloc[:,i], predicted[:,i],average='weighted')
    print(metrics[c],'\n')
#     print('Accuracy: ', accuracy_score(y_test.iloc[:,i],predicted[:,i]), '\n')

related
(0.8039308985488353, 0.8158905793760566, 0.8068579437978279, None) 

request
(0.8839826617145664, 0.8905793760565545, 0.8782228813214725, None) 

offer
(0.9886599644708849, 0.9943138158905793, 0.9914788300550278, None) 

aid_related
(0.7505559258450304, 0.7508836637467343, 0.7451252901183969, None) 

medical_help
(0.8941680059607275, 0.9207007837713231, 0.8945536024370843, None) 

medical_products
(0.9429874708159784, 0.9531274012601814, 0.9373033818117968, None) 

search_and_rescue
(0.9641048955825351, 0.9718764407561088, 0.9614632852224217, None) 

security
(0.9716256779900141, 0.983556170278162, 0.9761516335020234, None) 

military
(0.9511569449424835, 0.9657292146918703, 0.952571694043082, None) 

child_alone
(1.0, 1.0, 1.0, None) 

water
(0.9533810578311017, 0.9575841401567543, 0.9492644724774246, None) 

food
(0.930175915241179, 0.9351467650222837, 0.9290344835668807, None) 

shelter
(0.9271239267951473, 0.9353004456738897, 0.9235448355603465, None) 

clothing
(0.98377542

In [13]:
# Test

metrics['related'][2] # f score of 'related' column

0.8068579437978279

In [14]:
# This is cleaner (but only gives values for positive class)
# print(classification_report(y_test, predicted, target_names=Y.columns))

In [15]:
# The following doesn't give averages either
category_names = Y.columns.values
def get_metrics_summary(y_test,y_pred):
    metrics_summary = pd.DataFrame(index = category_names, columns = ['accuracy', 'precision', 'recall', 'f-1_score'])
    for i, c in enumerate(category_names): 
        metrics_summary.loc[c,'accuracy'] = accuracy_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'precision'] = precision_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'recall'] = recall_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'f-1_score'] = fbeta_score(y_test.iloc[:,i],y_pred[:,i],beta=1)

    metrics_summary.loc['average'] = metrics_summary.mean(axis=0)
    
    return metrics_summary

In [16]:
# metrics_summary_1 = get_metrics_summary(y_test,predicted)
# metrics_summary_1

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000002817113BCA8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

In [19]:
# parameters = {
#     'clf__estimator__max_depth':[2],
#     'clf__estimator__n_estimators':[20,50]
#     }

# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__ngram_range': ((1, 1), (1,2)),
#     'vect__max_features': (None, 5000,10000),
#     'tfidf__use_idf': (True, False)
# }

# parameters = {
#     'vect__ngram_range': ((1, 1), (1, 2)),
#     'clf__estimator__bootstrap': (True, False)
# }

# parameters = { # try next
#     'clf__estimator__n_estimators': [50, 100, 150],
#     'clf__estimator__min_samples_split': [2, 3, 4]
# }

# parameters = { # try
#         'clf__estimator__n_estimators': [100, 200],
#         'clf__estimator__learning_rate': [0.1, 0.3]
#     }

parameters = { # try next
    'clf__estimator__n_estimators': [200,300],
    'clf__estimator__min_samples_split': [2,3]
}

# BEST PARAMS for [50,100] and [2,3] was {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}****
# BEST PARAMS for [100,200] and [2,3] was {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}

# parameters = { # try next
# #     'clf__estimator__criterion': ['gini', 'entropy'], # In this, keep gini or entropy.
#     'clf__estimator__max_depth': [2, 5], # Use only two [2,5,None]
#     'clf__estimator__n_estimators': [100, 200], # Use only two [10,20,50]
#     'clf__estimator__min_samples_leaf':[2, 5], # can be ignored [1,5,10]
# }

# scoring = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score), 'recall': make_scorer(recall_score)}

# cv = GridSearchCV(pipeline,param_grid=parameters,scoring=scoring,refit='accuracy') #If scoring not included, refit not needed and grid search would find best model based on estimator's score method, which is average accuracy

# cv = GridSearchCV(pipeline,param_grid=parameters)

scoring = make_scorer(f1_score, average='weighted')
# cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2, scoring = ‘f1_weighted’) # the higher the verbose the more information

# cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2, scoring = scoring)
cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2) # better results when leave default scoring

# with active_session():
cv.fit(X_train,y_train)
#     cv.fit(X_train_sample,y_train_sample)

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 53.6min finished
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 53.5min remaining:    0.0s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [20]:
y_pred_improved = cv.predict(X_test)

In [21]:
# import pickle

# with open('train_classifier.pkl', 'wb') as file:
#     pickle.dump(cv, file)

In [22]:
# with open('train_classifier.pkl', "rb") as input_file:
#     e = pickle.load(input_file)

In [23]:
# y_pred_best = e.predict(X_test)
# e.best_params_

In [24]:
# e.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
# print(classification_report(y_test, y_pred_improved, target_names=Y.columns))

In [26]:
# metrics_2 = get_metrics_summary(y_test,y_pred_improved)
# metrics_2

In [31]:
# **COMPLETE THIS FUNCTION**

def compare_metric(y_test,y_pred1,y_pred2,metric='f-1_score'):
    metrics_summary = pd.DataFrame(index = category_names, columns = ['metric_old', 'metric_new'])
    for i, c in enumerate(category_names):
        if metric=='accuracy':
            metrics_summary.loc[c,'metric_old'] = accuracy_score(y_test.iloc[:,i],y_pred1[:,i])
            metrics_summary.loc[c,'metric_new'] = accuracy_score(y_test.iloc[:,i],y_pred2[:,i])
        elif metric=='precision':
#             metrics_summary.loc[c,'metric_old'] = precision_score(y_test.iloc[:,i],y_pred1[:,i])
#             metrics_summary.loc[c,'metric_new'] = precision_score(y_test.iloc[:,i],y_pred2[:,i])
            metrics_summary.loc[c,'metric_old'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred1[:,i], average='weighted')[0] # 0 for 1st entry in tuple for precision
            metrics_summary.loc[c,'metric_new'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred2[:,i], average='weighted')[0]
        elif metric=='recall':
#             metrics_summary.loc[c,'metric_old'] = recall_score(y_test.iloc[:,i],y_pred1[:,i])
#             metrics_summary.loc[c,'metric_new'] = recall_score(y_test.iloc[:,i],y_pred2[:,i])
            metrics_summary.loc[c,'metric_old'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred1[:,i], average='weighted')[1] # 1 for 2nd entry in tuple for recall
            metrics_summary.loc[c,'metric_new'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred2[:,i], average='weighted')[1]
        elif metric=='f-1_score':
#             metrics_summary.loc[c,'metric_old'] = fbeta_score(y_test.iloc[:,i],y_pred1[:,i],beta=1)
#             metrics_summary.loc[c,'metric_new'] = fbeta_score(y_test.iloc[:,i],y_pred2[:,i],beta=1)
            metrics_summary.loc[c,'metric_old'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred1[:,i], average='weighted')[2] # 2 for 3rd entry in tuple for f-score
            metrics_summary.loc[c,'metric_new'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred2[:,i], average='weighted')[2]
        
    metrics_summary['improved'] = metrics_summary['metric_new']>=metrics_summary['metric_old']
    metrics_summary.loc['sum'] = metrics_summary.sum()

#     metrics_summary.loc['average'] = metrics_summary.mean(axis=0)
    
    return metrics_summary

In [32]:
f1_comparison = compare_metric(y_test,predicted,y_pred_improved)
f1_comparison

,metric_old,metric_new,improved
related,0.806858,0.806712,0.0
request,0.878223,0.889369,1.0
offer,0.991479,0.991479,1.0
aid_related,0.745125,0.783931,1.0
medical_help,0.894554,0.891289,0.0
medical_products,0.937303,0.934633,0.0
search_and_rescue,0.961463,0.959795,0.0
security,0.976152,0.976311,1.0
military,0.952572,0.952447,0.0
child_alone,1,1,1.0


In [33]:
recall_comparison = compare_metric(y_test,predicted,y_pred_improved,metric='recall')
recall_comparison

,metric_old,metric_new,improved
related,0.815891,0.824804,1.0
request,0.890579,0.8998,1.0
offer,0.994314,0.994314,1.0
aid_related,0.750884,0.785308,1.0
medical_help,0.920701,0.920854,1.0
medical_products,0.953127,0.952974,0.0
search_and_rescue,0.971876,0.971876,1.0
security,0.983556,0.983864,1.0
military,0.965729,0.966805,1.0
child_alone,1,1,1.0


In [34]:
precision_comparison = compare_metric(y_test,predicted,y_pred_improved,metric='precision')
precision_comparison

,metric_old,metric_new,improved
related,0.803931,0.811942,1.0
request,0.883983,0.895284,1.0
offer,0.98866,0.98866,1.0
aid_related,0.750556,0.784009,1.0
medical_help,0.894168,0.893822,0.0
medical_products,0.942987,0.947538,1.0
search_and_rescue,0.964105,0.968411,1.0
security,0.971626,0.973762,1.0
military,0.951157,0.958716,1.0
child_alone,1,1,1.0


In [35]:
accuracy_comparison = compare_metric(y_test,predicted,y_pred_improved,metric='accuracy')
accuracy_comparison

,metric_old,metric_new,improved
related,0.815891,0.824804,1.0
request,0.890579,0.8998,1.0
offer,0.994314,0.994314,1.0
aid_related,0.750884,0.785308,1.0
medical_help,0.920701,0.920854,1.0
medical_products,0.953127,0.952974,0.0
search_and_rescue,0.971876,0.971876,1.0
security,0.983556,0.983864,1.0
military,0.965729,0.966805,1.0
child_alone,1,1,1.0


In [29]:
cv.best_params_

# 12/1/20 with all data, 26 columns same or improved; best estimator 2 min samples split and 300 n estimators

# 0.948995	0.592643	0.208286	0.258425 with n_estimators = 200 and min_samples_split =2 as best from [100,200] and [2,3]

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 300}

In [30]:
with open('tuned_model.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [ ]:
# a=1
# with active_session():
#     while a==1:
#         b=2

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
# Try using different classifier

pipeline2 = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [37]:
pipeline2.fit(X_train,y_train)

predicted2 = pipeline2.predict(X_test)

In [38]:
# metrics_adaboost = get_metrics_summary(y_test,predicted2)
# metrics_adaboost

metrics_adaboost = dict()
for i, c in enumerate(category_names): 
    print(c)
    metrics_adaboost[c] = precision_recall_fscore_support(y_test.iloc[:,i], predicted2[:,i],average='weighted')
    print(metrics_adaboost[c],'\n')

related
(0.7728584751585439, 0.7931458429383741, 0.7478815661754741, None) 

request
(0.8852936377947437, 0.892730905179038, 0.8837823374445046, None) 

offer
(0.988650390522976, 0.9926233287229138, 0.9906328762667364, None) 

aid_related
(0.7616151032292771, 0.7616413093591516, 0.7564878269793193, None) 

medical_help
(0.9167875822401291, 0.929767942216075, 0.9185208615055355, None) 

medical_products
(0.9502375436125062, 0.9575841401567543, 0.9514962250384766, None) 

search_and_rescue
(0.9629842186712023, 0.9715690794528968, 0.9640122419640273, None) 

security
(0.9701186732548694, 0.9798678346396189, 0.9747066070859715, None) 

military
(0.9637146050362115, 0.9692638696788074, 0.9653588781069248, None) 

child_alone
(1.0, 1.0, 1.0, None) 

water
(0.9628501027505798, 0.9649608114338405, 0.963616252762809, None) 

food
(0.9413329784997714, 0.9445212847702474, 0.9419109810238306, None) 

shelter
(0.9394587793786485, 0.9448286460734594, 0.9400975904556849, None) 

clothing
(0.986356609

In [42]:
f1_comparison2 = compare_metric(y_test,y_pred_improved,predicted2)
print('F1-SCORE')
print(f1_comparison2,'\n')
recall_comparison2 = compare_metric(y_test,y_pred_improved,predicted2,metric='recall')
print('RECALL')
print(recall_comparison2,'\n')
precision_comparison2 = compare_metric(y_test,y_pred_improved,predicted2,metric='precision')
print('PRECISION')
print(precision_comparison2,'\n')
accuracy_comparison2 = compare_metric(y_test,y_pred_improved,predicted2,metric='accuracy')
print('ACCURACY')
print(accuracy_comparison2)

F1-SCORE
                       metric_old metric_new  improved
related                  0.806712   0.747882       0.0
request                  0.889369   0.883782       0.0
offer                    0.991479   0.990633       0.0
aid_related              0.783931   0.756488       0.0
medical_help             0.891289   0.918521       1.0
medical_products         0.934633   0.951496       1.0
search_and_rescue        0.959795   0.964012       1.0
security                 0.976311   0.974707       0.0
military                 0.952447   0.965359       1.0
child_alone                     1          1       1.0
water                    0.958213   0.963616       1.0
food                      0.94177   0.941911       1.0
shelter                  0.925494   0.940098       1.0
clothing                  0.98305    0.98637       1.0
money                    0.966831   0.973378       1.0
missing_people           0.983663   0.987398       1.0
refugees                 0.947586   0.957449       1.0
d

F1-score improved, and precision also improved marginally, but recall and accuracy got marginally worse. So, untuned AdaBoost might be better. Let's see if tuning will improve it.

In [43]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000002817113BCA8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None)

In [56]:
# parameters2 = {
#     'clf__estimator__n_estimators': [50,100],
#     'clf__estimator__learning_rate': [0.5, 1.0]
# }
# 100 and 0.5 were best
# parameters2 = {
#     'clf__estimator__n_estimators': [100,200],
#     'clf__estimator__learning_rate': [0.25,0.5]
# }
# 200 and 0.5 best; better than previous
parameters2 = {
    'clf__estimator__n_estimators': [200,300],
    'clf__estimator__learning_rate': [0.25,0.5]
}
# 200 and 0.5 is still best

cv2 = GridSearchCV(pipeline2,param_grid=parameters2, n_jobs=-1, verbose=2) # the higher the verbose the more information

# with active_session():
cv2.fit(X_train,y_train)

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 27.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 27.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [57]:
y_pred2 = cv2.predict(X_test)

In [58]:
cv2.best_params_

{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 200}

In [59]:
# metrics_adaboost_new = get_metrics_summary(y_test,y_pred2)
# metrics_adaboost_new

metrics_adaboost_new = dict()
for i, c in enumerate(category_names): 
    print(c)
    metrics_adaboost_new[c] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred2[:,i],average='weighted')
    print(metrics_adaboost_new[c],'\n')

related
(0.793911730471203, 0.8106654372214538, 0.7858434877801782, None) 

request
(0.8942855215461295, 0.8994928538497003, 0.8897633964928066, None) 

offer
(0.9886477738163532, 0.9921622867680959, 0.9904019124260711, None) 

aid_related
(0.7737622497884402, 0.7714768710619333, 0.7653130068092164, None) 

medical_help
(0.9145624554882792, 0.9289995389580452, 0.9135920190870976, None) 

medical_products
(0.9481386614576659, 0.9566620562471185, 0.9474536075999819, None) 

search_and_rescue
(0.9643502834077818, 0.9721838020593208, 0.9631384106333494, None) 

security
(0.9719048143896601, 0.9826340863685262, 0.9762000391619489, None) 

military
(0.9635859033911918, 0.9698785922852313, 0.9646899599443973, None) 

child_alone
(1.0, 1.0, 1.0, None) 

water
(0.963611712295226, 0.9652681727370525, 0.9642778757564696, None) 

food
(0.9452743010893281, 0.9474412171507607, 0.9459928513920635, None) 

shelter
(0.9441013541781823, 0.9486706623636084, 0.9439908206307707, None) 

clothing
(0.9868452

In [60]:
f1_comparison2_new = compare_metric(y_test,y_pred_improved,y_pred2)
print('F1-SCORE')
print(f1_comparison2_new,'\n')
recall_comparison2_new = compare_metric(y_test,y_pred_improved,y_pred2,metric='recall')
print('RECALL')
print(recall_comparison2_new,'\n')
precision_comparison2_new = compare_metric(y_test,y_pred_improved,y_pred2,metric='precision')
print('PRECISION')
print(precision_comparison2_new,'\n')
accuracy_comparison2_new = compare_metric(y_test,y_pred_improved,y_pred2,metric='accuracy')
print('ACCURACY')
print(accuracy_comparison2_new)

C:\Users\dagus\Anaconda3\envs\udacity\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-SCORE
                       metric_old metric_new  improved
related                  0.806712   0.785843       0.0
request                  0.889369   0.889763       1.0
offer                    0.991479   0.990402       0.0
aid_related              0.783931   0.765313       0.0
medical_help             0.891289   0.913592       1.0
medical_products         0.934633   0.947454       1.0
search_and_rescue        0.959795   0.963138       1.0
security                 0.976311     0.9762       0.0
military                 0.952447    0.96469       1.0
child_alone                     1          1       1.0
water                    0.958213   0.964278       1.0
food                      0.94177   0.945993       1.0
shelter                  0.925494   0.943991       1.0
clothing                  0.98305   0.987022       1.0
money                    0.966831   0.973153       1.0
missing_people           0.983663   0.986791       1.0
refugees                 0.947586   0.958814       1.0
d

27,23,17,23 tuned with 0.5/100
27,20,21,20 tuned with 0.5/200 (then 26,20,21,20)

### 9. Export your model as a pickle file

In [61]:
with open('tuned_model_2.pkl', 'wb') as file:
    pickle.dump(cv2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.